<a href="https://colab.research.google.com/github/yashtriyar/AlgoBook/blob/master/Screening_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.5/241.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.5/37.5 MB 23.3 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/qiskit-research/qiskit-research.git
%cd qiskit-research
!pip install .

Cloning into 'qiskit-research'...
remote: Enumerating objects: 5063, done.
remote: Counting objects: 100% (455/455), done.
remote: Compressing objects: 100% (264/264), done.
remote: Total 5063 (delta 266), reused 330 (delta 185), pack-reused 4608
Receiving objects: 100% (5063/5063), 57.12 MiB | 23.60 MiB/s, done.
Resolving deltas: 100% (3199/3199), done.
/content/qiskit-research
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/qiskit-research
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 15.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.8/383.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [4]:
from qiskit_research.protein_folding.interactions.random_interaction import (
    RandomInteraction,
)
from qiskit_research.protein_folding.interactions.miyazawa_jernigan_interaction import (
    MiyazawaJerniganInteraction,
)
from qiskit_research.protein_folding.peptide.peptide import Peptide
from qiskit_research.protein_folding.protein_folding_problem import (
    ProteinFoldingProblem,
)

from qiskit_research.protein_folding.penalty_parameters import PenaltyParameters

from qiskit.utils import algorithm_globals, QuantumInstance

algorithm_globals.random_seed = 23
main_chain = "APRLRFY"
side_chains = [""] * 7
random_interaction = RandomInteraction()
mj_interaction = MiyazawaJerniganInteraction()
penalty_back = 10
penalty_chiral = 10
penalty_1 = 10

penalty_terms = PenaltyParameters(penalty_chiral, penalty_back, penalty_1)
peptide = Peptide(main_chain, side_chains)
protein_folding_problem = ProteinFoldingProblem(peptide, mj_interaction, penalty_terms)
qubit_op = protein_folding_problem.qubit_op()

In [6]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit.algorithms.minimum_eigensolvers import SamplingVQE
from qiskit import execute, Aer
from qiskit.primitives import Sampler
from typing import List, Dict, Any, Callable, Tuple, Optional, Union
import numpy as np
from qiskit import QuantumCircuit
from qiskit.opflow import OperatorBase, PauliSumOp
from qiskit.utils import QuantumInstance
from time import time

counts = []
measurements = []
values = []


class CustomSampler(Sampler):
    def __init__(self):
        super().__init__()

    def sample(self, circuit: QuantumCircuit, repetitions: int = 1) -> List[Dict[str, Any]]:
        qc = circuit.copy()
        qc.measure_all()
        backend = Aer.get_backend('qasm_simulator')
        job = execute(qc, backend, shots=repetitions)
        result = job.result()
        counts = result.get_counts(qc)
        measurements = []
        for bitstring, count in counts.items():
            measurements.append({'bitstring': bitstring, 'count': count})
        return measurements


def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)
    measurements.append({'parameters': parameters, 'mean': mean, 'std': std})


# set classical optimizer
optimizer = COBYLA(maxiter=50)

# set variational ansatz
ansatz = RealAmplitudes(reps=1)

# set custom sampler
sampler = CustomSampler()

# initialize VQE using CVaR with alpha = 0.1
vqe = SamplingVQE(
    sampler=sampler,
    ansatz=ansatz,
    optimizer=optimizer,
    aggregation=0.1,
    callback=store_intermediate_result,
)

# Assuming you have defined 'qubit_op' somewhere

raw_result = vqe.compute_minimum_eigenvalue(qubit_op)
print(raw_result)
print("Measurements:")
# for measurement in measurements:
#     print(measurement)
for index, measurement in enumerate(measurements, start=1):
    print(index, measurement)

SamplingMinimumEigensolverResult:
	Eigenvalue: -1.424999999999614
	Best measurement
: {'state': 165, 'bitstring': '010100101', 'value': (-1.4249999999996135+0j), 'probability': 0.0038907910604927}

Measurements:
1 {'parameters': [0.7838423033219764, 1.5249570586597203, -3.0271296314251086, 0.9692262567242826, 0.2642867883228641, 2.2075406912327287, 2.758502107235997, -3.0610226240215836, 2.062947537861036, -1.5499117056945564, 0.783563408369476, 1.6613895556624305, 2.18024157641206, 2.7683897737290772, 0.8463792877027871, 2.2569738080876025, -0.014459268936528602, -1.6753431921616952], 'mean': (-0.010551497542092725+0j), 'std': {}}
2 {'parameters': [1.7838423033219764, 1.5249570586597203, -3.0271296314251086, 0.9692262567242826, 0.2642867883228641, 2.2075406912327287, 2.758502107235997, -3.0610226240215836, 2.062947537861036, -1.5499117056945564, 0.783563408369476, 1.6613895556624305, 2.18024157641206, 2.7683897737290772, 0.8463792877027871, 2.2569738080876025, -0.014459268936528602, -